Inspired by: https://www.drivendata.org/competitions/217/cdc-fall-narratives/community-code/13/

Now we can use a medical dictionary (e.g. https://medical-dictionary.thefreedictionary.com/) and lots of googling to gather what some of the main terms mean. With this we can create a dictionary that we can then use to "translate" our narratives and make them clearer

In [1]:
import pandas as pd
import re

In [2]:
medical_terms = {
    "&": "and",
    "***": "",
    ">>": "clinical diagnosis",
    "@": "at",
    "+": "with",
    "?": "unknown if",
    "abd": "abdomen",
    "af": "accidental fall",
    "afib": "atrial fibrillation",
    "aki": "acute kidney injury",
    "am": "morning",
    "ams": "altered mental status",
    "bac": "blood alcohol content",
    "bal": "blood alcohol level,",
    "biba": "brought in by ambulance",
    "c/o": "complains of",
    "chi": "closed-head injury",
    "clsd": "closed",
    "cpk": "creatine phosphokinase",
    "cva": "cerebral vascular accident",
    "dx": "clinical diagnosis",
    "ecf": "extended-care facility",
    "er": "emergency room",
    "etoh": "ethyl alcohol",
    "eval": "evaluation",
    "fd": "fall detected",
    "ft": "foot",
    "fx": "fracture",
    "fxs": "fractures",
    "glf": "ground level fall",
    "h/o": "history of",
    "htn": "hypertension",
    "hx": "history of",
    "inj": "injury",
    "inr": "international normalized ratio",
    "intox": "intoxication",
    "l": "left",
    "lac": "laceration",
    "loc": "loss of consciousness",
    "lt": "left",
    "mech": "mechanical",
    "mult": "multiple",
    "n.h.": "nursing home",
    "nh": "nursing home",
    "p/w": "presents with",
    "pm": "afternoon",
    "pt": "patient",
    "pta": "prior to arrival",
    "pts": "patient's",
    "px": "physical examination", # not "procedure",
    "r": "right",
    "r/o": "rules out",
    "rt": "right",
    "s'd&f": "slipped and fell",
    "s/p": "after",
    "sah": "subarachnoid hemorrhage",
    "sdh": "acute subdural hematoma",
    "sts": "sit-to-stand",
    "t'd&f": "tripped and fell",
    "tr": "trauma",
    "uti": "urinary tract infection",
    "w/": "with",
    "w/o": "without",
    "wks": "weeks"
}

In [7]:
def add_space_after(text, target=","):
    replaced_text = re.sub(r'({})(\S)'.format(re.escape(target)), r'{} \2'.format(target),text)
    return replaced_text

def add_space_before(text, target=","):
    replaced_text = re.sub(r'(\S)({})'.format(re.escape(target)), r'\1 {}'.format(target), text)
    return replaced_text

# cleanning
def clean_narrative(text):
    # lowercase everything
    if pd.isna(text):
        return "NA"
    else:     
        text = text.lower()
        
        # unglue DX
        regex_dx = r"([ˆ\W]*(dx)[ˆ\W]*)"
        text = re.sub(regex_dx, r". dx: ", text)
    
        # remove age and sex identifications
        ## regex to capture age and sex (not perfect but captures almost all of the cases)
        regex_age_sex = r"(\d+)\s*?(yof|yf|yo\s*female|yo\s*f|yom|ym|yo\s*male|yo\s*m)"
        age_sex_match = re.search(regex_age_sex, text)
    
        ## format age and sex
        if age_sex_match:
            age = age_sex_match.group(1)
            sex = age_sex_match.group(2)
            
            # probably not best practice but it works with this data
            if "f" in sex:
                #text = text.replace(age_sex_match.group(0), f"{age} years old female")
                text = text.replace(age_sex_match.group(0), f"patient")
            elif "m" in sex:
                #text = text.replace(age_sex_match.group(0), f"{age} years old male")
                text = text.replace(age_sex_match.group(0), f"patient")
        
        # translate medical terms
        for term, replacement in medical_terms.items():
            if term == "@" or term == ">>" or term == "&" or term == "***" or term == "+" or term == "?":
                pattern = fr"({re.escape(term)})"
                text = re.sub(pattern, f" {replacement} ", text) # force spaces around replacement
                
            else:
                pattern = fr"(?<!-)\b({re.escape(term)})\b(?!-)"
                text = re.sub(pattern, replacement, text)
                     
        text = add_space_after(text, target=",") 
        text = add_space_after(text, target=";") 
        text = add_space_after(text, target=":") 
        text = add_space_before(text, target="--")  
        text = add_space_after(text, target="--")
        text = add_space_before(text, target="/")  
        text = add_space_after(text, target="/")
        text = re.sub("-", " ", text)
        text = re.sub("\.", " ", text)
        text = re.sub(r"([0-9]+(\.[0-9]+)?)",r" \1 ", text).strip()
        # remove extra white spaces
        
        text = re.sub(r'\s+', ' ', text)
    
        return text.upper()

In [8]:
text = '72 YOF SLIPPED AND FELL ON THE FLOOR THIS AM>>L-3, L-4 FRACTURE, +LOC, RT RIB FRACTURES X 3.'
print("Original text:", text)
print("Clean text:", clean_narrative(text))
print()

Original text: 72 YOF SLIPPED AND FELL ON THE FLOOR THIS AM>>L-3, L-4 FRACTURE, +LOC, RT RIB FRACTURES X 3.
Clean text: PATIENT SLIPPED AND FELL ON THE FLOOR THIS MORNING CLINICAL DIAGNOSIS L 3 , L 4 FRACTURE, WITH LOSS OF CONSCIOUSNESS, RIGHT RIB FRACTURES X 3



In [9]:
import pandas as pd
df = pd.read_csv(
    "primary_data.csv",
    # set columns that can be null to nullable ints
    dtype={"body_part_2": "Int64", "diagnosis_2": "Int64"},
)
df.head()

,cpsc_case_number,narrative,treatment_date,age,sex,race,other_race,hispanic,diagnosis,other_diagnosis,...,body_part,body_part_2,disposition,location,fire_involvement,alcohol,drug,product_1,product_2,product_3
0,190103269,94YOM FELL TO THE FLOOR AT THE NURSING HOME ON...,2019-01-01,94,1,0,NaN,0,62,NaN,...,75,<NA>,4,5,0,0,0,1807,0,0
1,190103270,86YOM FELL IN THE SHOWER AT HOME AND SUSTAINED...,2019-01-01,86,1,0,NaN,0,62,NaN,...,75,<NA>,4,1,0,0,0,611,0,0
2,190103273,87YOF WAS GETTING UP FROM THE COUCH AND FELL T...,2019-01-01,87,2,0,NaN,0,53,NaN,...,32,<NA>,4,1,0,0,0,679,1807,0
3,190103291,67YOF WAS AT A FRIENDS HOUSE AND SLIPPED ON WA...,2019-01-01,67,2,0,NaN,0,57,NaN,...,33,<NA>,1,1,0,0,0,1807,0,0
4,190103294,70YOF WAS STANDING ON A STEP STOOL AND FELL OF...,2019-01-01,70,2,0,NaN,0,57,NaN,...,33,<NA>,1,1,0,0,0,620,0,0


In [10]:
df_narratives = df.iloc[:, 0:2]
df_narratives.head(2)

,cpsc_case_number,narrative
0,190103269,94YOM FELL TO THE FLOOR AT THE NURSING HOME ON...
1,190103270,86YOM FELL IN THE SHOWER AT HOME AND SUSTAINED...


In [11]:
df_narratives['narrative'] = df_narratives['narrative'].apply(lambda x: clean_narrative(x))

In [12]:
df_narratives['narrative'][100000]

'PATIENT HAS RIGHT FINGER INJURY AFTER PATIENT WAS TAKING CARE OF GRANDSON WHEN HE TRIPPED AND FELL INTO THE DOOR FINGER GOT CAUGHT ON DOOR JAMB AND BENT OUT OF POSITION CLINICAL DIAGNOSIS: CONTUSION OF RIGHT KNEE; DISLOCATION OF FINGER; CLOSED DISPLACED FRACTURE OF PROXIMAL PHALANX OF RIGHT LITTLE FINGER'

In [15]:
df_nn = df.drop('narrative', axis=1)

In [17]:
df_final = pd.merge(df_narratives, df_nn, on="cpsc_case_number", how="left")

In [19]:
len(df_final)

115128

In [21]:
df_final.to_csv("corrected_narrative_primary.csv", index=False)